## <center> **Проект: Построение модели, предсказывающей рейтинг отелей**

Одна из проблем компании Booking — это нечестные отели, которые накручивают себе рейтинг. 

Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.

***Цель проекта***: построить модель на основе предложенного алгоритма машинного обучения, которая будет предсказывать рейтинг отеля.

***Задачи проекта***:

1 Ознакомиться с реализацией примера данной модели на площадке kaggle.com

2 Проанализировать входные данные

3 Выполнить подготовку данных, которые будут использованы для обучения модели

4 Проверить эффективность полученной модели, используя метрику MAPE

5 Принять участие в соревнованиях на площадке kaggle.com

6 Разместить проект на платформе GitHub

***В исходном датасете представлены следующие признаки***:

hotel_address — адрес отеля;

review_date — дата, когда рецензент разместил соответствующий отзыв;

average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;

hotel_name — название отеля;

reviewer_nationality — страна рецензента;

negative_review — отрицательный отзыв, который рецензент дал отелю;

review_total_negative_word_counts — общее количество слов в отрицательном отзыв;

positive_review — положительный отзыв, который рецензент дал отелю;

review_total_positive_word_counts — общее количество слов в положительном отзыве.

reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;

total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;

total_number_of_reviews — общее количество действительных отзывов об отеле;

tags — теги, которые рецензент дал отелю;

days_since_review — количество дней между датой проверки и датой очистки;

additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.

lat — географическая широта отеля;

lng — географическая долгота отеля.

### 1 Подготовка данных

#### 1.1 Установка пакетов, импорт библиотек

In [ ]:
# установим для возможности проведения анализа тональности отзывов
#!pip install nltk

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os # для работы с файловой системой и ОС
import re
import numpy as np 
import pandas as pd 

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# импортируем для проведения анализа тональности отзывов
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from pandas_profiling import ProfileReport # библиотека для создания подробного отчёта по данным

from geopy.distance import geodesic as GD # для расчета расстояния по координатам

from scipy.stats import shapiro # для проведения теста Шапиро-Уилка

import category_encoders as ce # импорт для работы с кодировщиком
from sklearn.preprocessing import OrdinalEncoder

from sklearn.feature_selection import f_classif # для определения значимости признаков
from sklearn.feature_selection import chi2 # для определения значимости признаков

# испортируем библиотеки машинного обучения:
from sklearn.model_selection import train_test_split # для разделения датасета

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#### 1.2 Получение списка файлов для обработки

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### 1.3 Определение констант

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

#### 1.4 Фиксация версии пакетов

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

#### 1.5 Подгрузка данных

In [ ]:
DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

#### 1.6 Объединение датасетов

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort = False).reset_index(drop = True) # объединяем

### 2 Исследование структуры данных

#### 2.1 Ознакомимся с признаками, их структурой в первоначальных датасетах

In [ ]:
df_train.head(2)

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(2)

#### 2.2 Выведем информацию о типах данных, количестве непустых значений в первоначальных датасетах

In [ ]:
df_train.info()

In [ ]:
df_test.info()

***Вывод***. В данных присутствуют пропуски в столбцах "lat", "lng".

Отличные от числовых признаки (в представленном формате не могут быть использованы в модели) необходимо будет в дальнейшем проанализировать и преобразовать. 

In [ ]:
sample_submission.info()

#### 2.3 Ознакомимся с основной статистическими информацией о столбцах в объединенном датасете

In [ ]:
data.describe(include = 'all') 

### 3 Исследование и проектирование признаков

#### 3.1 Выявление взаимосвязи между исходными признаками

In [ ]:
plt.rcParams['figure.figsize'] = (10,6)
sns.heatmap(data.drop(['sample'], axis = 1).corr(), annot = True)

***Вывод***. Визуализация корреляции позволяет наглядно оценить наличие сильной взаимосвязи между двумя признаками "Total_number_of_reviews" и "Additional_number_of_scoring", в связи с этим один из признаков подлежит удалению

Также хотелось бы отметить, что изначально представленные признаки имеют слабую связь с целевой переменной reviewer_scorе. В связи с этим необходимо провести преобразование признаков и добавление в датасет новых признаков

In [ ]:
data = data.drop(["additional_number_of_scoring"], axis = 1)

#### 3.2 Преобразование и создание новых признаков

#### <center> *Признаки с типом "object"*

In [ ]:
object_columns = list(data.select_dtypes(["object"]).columns)

print(f"Признаки с типом 'object': {object_columns}")

#### 3.2.1 Признак "hotel_address"

In [ ]:
# Рассмотрим структуру данных, содержащих адреса отелей
hotel_address = pd.Series(data['hotel_address'].unique())
display(hotel_address)

print()


# Выделим название города, страны из адреса отеля
def select_city_country(adress):
    adress = adress.replace("United Kingdom", "UnitedKingdom") # заменим - для дальнейшего корректного выделения названия страны  
    adress_list = [word for word in adress.split(' ') if word.isalpha()] # используя регулярные выражения, выделим в адресе все слова, не включающие цифры
    return (adress_list[len(adress_list) - 2], adress_list[len(adress_list) - 1]) # возвращаем кортеж (Город, Страна)


# Создадим новые признаки, выделим уникальные значения
data["hotel_country"] = data["hotel_address"].apply(lambda x: select_city_country(x)[1])
data["hotel_country"] = data["hotel_country"].replace("UnitedKingdom", "United Kingdom") # приведем к привычному написанию
display(data["hotel_country"].unique())

print()

data["hotel_city"] = data["hotel_address"].apply(lambda x: select_city_country(x)[0])
display(data["hotel_city"].unique())

***Вывод.*** В каждой стране отели расположены только в одном из городов, в связи с этим в дальнейшем после преобразования всех признаков один из признаков hotel_country, hotel_city будет удален

#### 3.2.2 Признак "review_date"

In [ ]:
# Переведем столбец с информацией о дате ("review_date") в формат datetime
data["review_date"] = pd.to_datetime(data["review_date"], format = '%m/%d/%Y')

# Создадим новые признаки: год, месяц и день недели, когда рецензент разместил соответствующий отзыв
data["review_year"] = data["review_date"].dt.year
data['review_month'] = data["review_date"].dt.month 
data['review_weekday'] = data["review_date"].dt.dayofweek 

#### 3.2.3 Признак 'hotel_name'

Для данного признака преобразования не проводились в связи с тем, что часть названий отелей содержит опечатки, указана некорректно.

Признак 'hotel_name' будет в дальнейшем удален.

#### 3.2.4 Признак 'reviewer_nationality'

In [ ]:
# Удалим пробелы в начале и в конце строк 
data["reviewer_nationality"] = data["reviewer_nationality"].apply(lambda x: x.strip())

# Проверим есть ли в наших данных рецензенты, которые являются резидентами страны, в которой находится отель
resident = data[data["reviewer_nationality"] == data["hotel_country"]]

if resident.shape[0] != 0:
    print(f'Количество резидентов: {resident.shape[0]} человек')
else:
    print('Резидентов не найдено')
    
# Создадим новый признак, в котором отразим, является ли человек, оставивший отзыв, резидентом страны, в которой находится отель
data["resident"] = data['reviewer_nationality'] == data['hotel_country']
data["resident"] = data["resident"].apply(lambda x: 1 if x == True else 0)

In [ ]:
#Рассмотри особенности распределения данных в признаке "reviewer_nationality"
print(f'Количество уникальных значений для призака "reviewer_nationality": {data["reviewer_nationality"].nunique()}')

# Выведем распределение топ-15 самых часто встречающихся названий стран рецензентов 
data["reviewer_nationality"].value_counts(normalize = True).nlargest(15)

***Вывод***. 
Наибольшое количество рецензентов - из United Kingdom. 

В соответствии с полученными данными сгруппируем их следующим образом: выделим топ-10 стран проживания рецензентов в отдельные группы, остальные страны объединим их в группу "Other".

In [ ]:
top_10 = data["reviewer_nationality"].value_counts(normalize = True).nlargest(10).index # список стран топ-4 по количеству отзывов

data["reviewer_nationality"] = data["reviewer_nationality"].apply(lambda x: x if x in top_10 else 'Other')

# Выведем полученное распределение названий стран рецензентов по сформированным группам
data["reviewer_nationality"].value_counts(normalize = True)

#### 3.2.5 Признаки 'negative_review', 'positive_review'

In [ ]:
#Преобразуем признаки, приведя значения к нижнему регистру и удалив пробелы в начале и в конце строки
data["negative_review"] = data['negative_review'].str.lower().str.strip()
data["positive_review"] = data['positive_review'].str.lower().str.strip()

In [ ]:
# Выведем топ-60 негативных отзывов по количеству значений (более 100)
display(data["negative_review"].value_counts().reset_index().head(60))

***Вывод***. Самыми часто встречающимися негативными  отзывами являются "no negative". Программа воспринимает такие отзывы, размещенные в признаке "negative_review", в качестве негативных, что является неверным в данном контексте употребления (в этих отзывах другой смысловой оттенок). 

Отзывы с подобными значениями, которые не являются негативными по своему содержанию, но размещены в данном столбце, с количеством значений более 100, занесем в список с целью дальнейшего преобразования 

In [ ]:
# Создадим дополнительные признаки, в которых отметим,
# что отзыв по своему содержанию не является негативным или положительным
data['no_negative_review'] = 0
data['no_positive_review'] = 0

In [ ]:
# Если рецензент оставил поле для отрицательного отзыва пустым, то в таблице это поле заполнено как "No negative",
# что будет некорректно интерапретироваться при анализе отзывов, требует дальнейшего преобразования
data[data['review_total_negative_word_counts'] == 0]['negative_review'].value_counts()

In [ ]:
no_negative_list = ['nothing', 'no negative', 'n a', 'none', '', 'all good', 'i liked everything',
                    'nothing really', 'no complaints', 'nil', 'nothing at all', 'everything was good',
                    'na', 'nothing to dislike', 'liked everything', 'no', 'nothing it was perfect',
                    'can t think of anything', 'everything was perfect', 'nothing everything was perfect',
                    'absolutely nothing', 'nothing to dislike', 'non', 'nothing everything was great',
                    'everything was fine', 'all was good', 'nothing comes to mind',
                    'everything was great', 'nothing to complain about', 'nothing all good']

data.loc[data['negative_review'].isin(no_negative_list), 'no_negative_review'] = 1

# Заменим отзывы из no_negative_list на '' для корректной обработки именно отрицательных отзывов
data.loc[data['negative_review'].isin(no_negative_list), 'negative_review'] = ''




In [ ]:
# Если рецензент оставил поле для положительного отзыва пустым, то в таблице это поле заполнено как "No positive",
# что будет некорректно интерапретироваться при анализе отзывов, требует дальнейшего преобразования
data[data['review_total_positive_word_counts'] == 0]['positive_review'].value_counts()

In [ ]:
# Выведем топ-60 положительных отзывов по количеству значений
display(data["positive_review"].value_counts().reset_index().head(60))

***Вывод***. Самыми часто встречающимися позитивными отзывами являются "no positive". Программа воспринимает такие отзывы, размещенные в признаке "positive_review", в качестве позитивных, что является неверным в данном контексте употребления (в этих отзывах заложен другой смысловой оттенок). 

Отзывы с подобными значениями, которые не являются положительными по своему содержанию, но размещены в данном столбце, занесем в список с целью дальнейшего преобразования 

In [ ]:
no_positive_list = ['no positive', 'nothing']

data.loc[data['positive_review'].isin(no_positive_list), 'no_positive_review'] = 1

# Заменим отзывы из no_positive_list на '' для корректной обработки именно положительных отзывов
data.loc[data['positive_review'].isin(no_positive_list), 'positive_review'] = ''

In [ ]:
# С помощью библиотеки nltk переведем отзывы из текстового в числовое представление 
nltk.downloader.download('vader_lexicon')
sent_analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Получим общую информацию по каждому негативному отзыву (временный признак)
data["negative_score_united"] = data["negative_review"].apply(lambda x: sent_analyzer.polarity_scores(x))

# Выделим информацию в отдельные признаки
data["negative_review_neg"] = data["negative_score_united"].apply(lambda x: x["neg"])
data["negative_review_neu"] = data["negative_score_united"].apply(lambda x: x["neu"])
data["negative_review_pos"] = data["negative_score_united"].apply(lambda x: x["pos"])
data["negative_review_compound"] = data["negative_score_united"].apply(lambda x: x["compound"])


In [ ]:
# Получим общую информацию по каждому положительному отзыву (временный признак)
data["positive_score_united"] = data["positive_review"].apply(lambda x: sent_analyzer.polarity_scores(x))

# Выделим информацию в отдельные признаки
data["positive_review_neg"] = data["positive_score_united"].apply(lambda x: x["neg"])
data["positive_review_neu"] = data["positive_score_united"].apply(lambda x: x["neu"])
data["positive_review_pos"] = data["positive_score_united"].apply(lambda x: x["pos"])
data["positive_review_compound"] = data["positive_score_united"].apply(lambda x: x["compound"])

In [ ]:
# Удалим временные признаки
data = data.drop(["negative_score_united", "positive_score_united"], axis = 1)

#### 3.2.6 Признак 'tags'

In [ ]:
# Создадим множество и список для подсчета количества 
# уникальных тегов и общего количества тегов 
tags_tuple = set()
tags_list = list()

for index, value in data["tags"].items():
        tags = value
        tags = tags.replace("[' ", "").replace(" ']", '')
        tags = tags.split(" ', ' ")
        for i in tags:
            tags_tuple.add(i.strip())
            tags_list.append(i.strip())

print(f'Количество уникальных тегов: {len(tags_tuple)}')
print()
print(f'Общее количество тегов: {len(tags_list)}')

In [ ]:
# Выведем самые популярные теги, предварительно создав отдельный df
tags_df = pd.DataFrame(data = tags_list, columns = ["tags_name"])
tags_df = tags_df.value_counts().rename_axis('tags_name').reset_index(name = 'counts')
tags_df.head(60)


***Вывод.*** 

В топ-5 самых популярных тегов по количеству входят: "Leisure trip",  "Submitted from a mobile device", "Couple", "Stayed 1 night", "Stayed 2 nights"

In [ ]:
# Рассмотрим структуру данных, содержащих признак 'tags', создав временный признак
data['len_tags'] = data['tags'].apply(lambda x: len(x.split(',')))

print(f'Максимальное количество тегов в одной строке: {data["len_tags"].max()}')
print()
#Выведем несколько примеров для ознакомления
print(data[data['len_tags'] == data['len_tags'].max()]['tags'].iloc[0])
print()
print(data[data['len_tags'] == data['len_tags'].max()]['tags'].iloc[10])

In [ ]:
#Удалим временный признак 'len_tags'
data = data.drop(['len_tags'], axis = 1)

На основе полученных данных о структуре тегов выделим в отдельные признаки: путешествие с животными, цель поездки, состав путешественников, тип номера, количество ночей, доступ осуществляется с мобильного устройства

In [ ]:
#Преобразуем признак, приведя значения к нижнему регистру 
data["tags"] = data["tags"].str.lower()

In [ ]:
# Создадим новый признак "with_pet" на основании информации из тегов 
data['with_pet'] = data['tags'].apply(lambda x: 1 if "with a pet" in x else 0)

In [ ]:
# Выделим отдельно уникальные значения данных, содержащих слово "trip", 
# для проведения классификации тегов по цели поездки

type_trip = tags_df[tags_df['tags_name'].str.contains("trip")]
type_trip['tags_name'].unique()

In [ ]:
# Создадим функцию для получения информации из тегов о цели поездки
def select_trip(tags): 
    expression = re.findall(r'leisure trip|business trip', tags)
    for elem, word in enumerate(expression):
        return word

    
# Создадим новый признак "purpose_trip"
data['purpose_trip'] = data['tags'].apply(select_trip) 

print(f'Количество пропусков: {data[data["purpose_trip"].isna()].shape[0]}')

# Заполняем отсутствующие значения на 'other'
data['purpose_trip'] = data['purpose_trip'].fillna('other')


In [ ]:
# Создадим функцию для получения информации из тегов о составе путешественников
def select_travelers(tags): 
    expression = re.findall(r'couple|group|family with young children|family with older children|solo|with friends', tags)
    for elem, word in enumerate(expression):
        return word

    
# Создадим новый признак "travelers"
data['travelers'] = data['tags'].apply(select_travelers)
data['travelers'].unique()

In [ ]:
print(f'Количество пропусков: {data[data["travelers"].isna()].shape[0]}')

In [ ]:
# Выделим отдельно уникальные значения данных, содержащих слово "room", 
# для проведения классификации тегов по типу номера

type_room = tags_df[tags_df['tags_name'].str.contains("room")]
type_room['tags_name'].unique()

In [ ]:
# Создадим функцию для получения информации из тегов о типе номера
def select_type_room(tags):
    premiers = ["premier", "royal", "presidential"]
    luxuries = ['luxury', "privilege", "executive"]
    deluxes  = ['deluxe', "premium"] 
    larges   = ['quadruple', 'triple', 'large', 'duplex', "penthouse", "townhouse",
                "apartment", "apartments"]
    for premier in premiers:
        if premier in tags:
            return 'premier'
    for luxury in luxuries:
        if luxury in tags:
            return 'luxury'
    for deluxe in deluxes:
        if deluxe in tags:
            return 'deluxe'  
    for large in larges:
        if large in tags:
            return 'large'    
    if 'suite' in tags:
        return 'suite'
    if "superior" in tags:
        return 'superior'
    for large in larges:
        if large in tags:
            return 'large'
    if "family room" in tags:
        return 'family'
    if "comfort" in tags:
            return 'comfort'
    if 'standard' in tags:
            return 'standard'    
    if "studio" in tags:
            return 'studio'
    if 'single' in tags:
        return 'single'    
    if 'double' in tags or 'twin' in tags:
        return 'double'
    else:
        return 'other'

    
# Создадим новый признак "type_of_room"
data["type_of_room"] = data["tags"].apply(select_type_room)

In [ ]:
data["type_of_room"].unique()

В полученном признаке "type_of_room" пропусков нет

In [ ]:
# Создадим функцию для получения информации из тегов о количестве ночей, 
# проведенных в отеле(длительность пребывания)
def select_count_nights(tags):
    expression = re.findall(r'stayed\s([0-9]+)\snight', tags)
    for elem, num in enumerate(expression):
        return num

    
# Создадим новый признак "count_nights"    
data['count_nights'] = data['tags'].apply(select_count_nights)   

In [ ]:
print(f'Количество пропусков: {data[data["count_nights"].isna()].shape[0]}')
print()
print(f'Самое часто встречающееся значение: {data["count_nights"].mode()[0]}')

# Заполняем отсутствующие значения на 1
data['count_nights'] = data['count_nights'].fillna(1)

# Изменяем тип данных
data['count_nights'] = data['count_nights'].astype('int64')

In [ ]:
# Создадим новый признак "submitted_mobile_device" на основании информации из тегов 
data['submitted_mobile_device'] = data['tags'].apply(lambda x: 1 if "submitted from a mobile device" in x else 0)

#### 3.2.7 Признак 'days_since_review'

In [ ]:
# Приведем признак "days_since_review" к числовому типу
data['days_since_review'] = data['days_since_review'].apply(lambda x: x.split()[0]).astype('int64')


#### <center> *Признаки с типами "int64", "float64"*

In [ ]:
numeric_columns = list(data.select_dtypes(["int64", "float64"]).columns)

print(f"Признаки, с типами 'int64, float64': {numeric_columns}")

#### 3.2.8 Признаки 'review_total_negative_word_counts', 'review_total_positive_word_counts'

In [ ]:
# Проверим, соответствует ли количество слов в признаках 
# review_total_negative_word_counts, review_total_positive_word_counts 
# реальному количеству слов в отзывах

display(data[['negative_review', 
              'review_total_negative_word_counts', 
              'positive_review', 
              'review_total_positive_word_counts']].head())

***Вывод.*** Количество слов в указанных графах не соответствует количеству слов в отзывах

In [ ]:
# Скорректируем указанное выше несоответствие
data['review_total_negative_word_counts'] = data['negative_review'].apply(lambda x: len(x.split()))
data['review_total_positive_word_counts'] = data['positive_review'].apply(lambda x: len(x.split()))

display(data[['negative_review', 
              'review_total_negative_word_counts', 
              'positive_review', 
              'review_total_positive_word_counts']].head())

#### 3.2.9 Признак "total_number_of_reviews_reviewer_has_given"

In [ ]:
print(f'Минимальное значение для данного признака: {data["total_number_of_reviews_reviewer_has_given"].min()}')
print()
print(f'Максимальное значение для данного признака: {data["total_number_of_reviews_reviewer_has_given"].max()}')


In [ ]:
# Выделим группы активности рецензентов в зависимости от количества отзывов,
# оставленных ранее
display(data['total_number_of_reviews_reviewer_has_given'].groupby(
    pd.cut(data['total_number_of_reviews_reviewer_has_given'], 
           np.arange(0, 380, 25))).agg('count'))

In [ ]:
# Создадим новый признак "groups_number_reviews"
data['groups_number_reviews'] = pd.cut(data['total_number_of_reviews_reviewer_has_given'], np.arange(0, 380, 25))

#### 3.2.10 Признаки "lat", "lng"

Имеющиеся пропуски в данных признаках заполним средними значениями широты и долготы для каждого из городов

In [ ]:
# Рассчитаем средние значений широты и долготы для каждого из городов
display(data.groupby("hotel_city", as_index = False).agg(mean_lat = ("lat", "mean"), mean_lng = ("lng", "mean"))) 

In [ ]:
# Выведем информацию, в каких городах есть пропущенные значения широты и долготы
display(data[data["lat"].isnull()].groupby("hotel_city", as_index = False).agg(qty_null_lat = ("hotel_city", "count")))

display(data[data["lng"].isnull()].groupby("hotel_city", as_index = False).agg(qty_null_lng = ("hotel_city", "count")))

In [ ]:
# Заполним пропуски полученными средними значениями широты и долготы для каждого города
data.loc[(data["hotel_city"] == "Barcelona") & (data["lat"].isnull()), "lat"] = 41.389079
data.loc[(data["hotel_city"] == "Barcelona") & (data["lng"].isnull()), "lng"] = 2.169147

data.loc[(data["hotel_city"] == "Paris") & (data["lat"].isnull()), "lat"] = 48.863715
data.loc[(data["hotel_city"] == "Paris") & (data["lng"].isnull()), "lng"] = 2.326780

data.loc[(data["hotel_city"] == "Vienna") & (data["lat"].isnull()), "lat"] = 48.203362
data.loc[(data["hotel_city"] == "Vienna") & (data["lng"].isnull()), "lng"] = 16.367234

# Проверим наличие пропусков
print(f"Количество пропусков для признака 'lat': {data['lat'].isnull().sum()}")
print(f"Количество пропусков для признака 'lng': {data['lng'].isnull().sum()}")


Сделаем предположение о влиянии на отзыв такого признака, как удаленность отеля от центра города. Для этого с помощью библиотеки geopy найдем удаленность каждого из отелей от центра города (координаты центров городов - Time-In.ru), добавив соответствующий признак в датасет

In [ ]:
#!pip install geopy

In [ ]:
def get_city_center_coords(hotel_city):
    if hotel_city == "London":
        coords = (51.5085, -0.12574)
    elif hotel_city == "Paris":
        coords = (48.8534, 2.3488)
    elif hotel_city == "Amsterdam":
        coords = (52.374, 4.88969)
    elif hotel_city == "Milan":
        coords = (45.4643, 9.18951)
    elif hotel_city == "Vienna":
        coords = (48.2085, 16.3721)
    elif hotel_city == "Barcelona":
        coords = (41.3888, 2.15899)
    return coords


def remoteness(lat1, long1, hotel_city):
    coords_1 = (lat1, long1)
    coords_2 = get_city_center_coords(hotel_city)
    remouteless = GD(coords_1, coords_2).km
    return round(remouteless, 0)


data["distance_from_center"] = data.apply(lambda x: remoteness(x['lat'], x['lng'], x['hotel_city']), axis = 1)


#### 3.3 Очистка данных

#### 3.3.1 Выявление дубликатов

In [ ]:
mask = data.duplicated(subset = data.columns)
duplicates = data[mask]
print(f'Число найденных дубликатов: {duplicates.shape[0]}')

***Вывод.*** 

В связи с тем,что количество дубликатов в сравнении с общим числом строк составляет около 0,065% и по условию соревнования строки не могут быть удалены, оставим эти записи в датасете

#### 3.3.2 Поиск пропущенных значений

In [ ]:
data.isnull().sum()

Имеющиеся в данных пропуски были обработаны в каждом из признаков, соответственно датасет не содержит пропущенных значений

#### 3.3.3 Выявление и обработка выбросов

In [ ]:
# Определим, имеются ли в числовых признаках выбросы
fig, axes = plt.subplots(nrows = 3, ncols = 3, figsize = (15, 11))
fig. tight_layout (h_pad = 7) # отрегулируем интервал заголовков

boxplot = sns.boxplot(
    data = data, 
    x = 'review_total_negative_word_counts', 
    ax = axes[0,0]);
boxplot.set_title('total_negative_word_counts');

boxplot = sns.boxplot(
    data = data, 
    x = 'review_total_positive_word_counts', 
    ax = axes[0,1]);
boxplot.set_title('total_positive_word_counts');

boxplot = sns.boxplot(
    data = data, 
    x = 'average_score', 
    ax = axes[0,2]);
boxplot.set_title('average_score');

boxplot = sns.boxplot(
    data = data, 
    x = 'total_number_of_reviews_reviewer_has_given', 
    ax = axes[1,0]);
boxplot.set_title('total_number_of_reviews_reviewer_has_given');

boxplot = sns.boxplot(
    data = data, 
    x = 'total_number_of_reviews', 
    ax = axes[1,1]);
boxplot.set_title('total_number_of_reviews');

boxplot = sns.boxplot(
    data = data, 
    x = 'days_since_review', 
    ax = axes[1,2]);
boxplot.set_title('days_since_review');

boxplot = sns.boxplot(
    data = data, 
    x = 'lat', 
    ax = axes[2,0]);
boxplot.set_title('lat');

boxplot = sns.boxplot(
    data = data, 
    x = 'lng', 
    ax = axes[2,1]);
boxplot.set_title('lng');

# Для целевого признака 'reviewer_score' отфильтруем данные, 
# так как нули в оценке рецензента сделали бы интерпретацию некорректной
mask = data['sample'] == 1

boxplot = sns.boxplot(
    data = data[mask], 
    x = 'reviewer_score',
    ax = axes[2,2]);
boxplot.set_title('reviewer_score')

***Вывод.*** В приведенных визуализациях boxplot для числовых признаков отображены потенциальные выбросы, которые требуют дополнительного изучения.

Так,для признака "days_since_review" с помощью визуализации выбросы не обнаружены, для признаков "lat", "lng" - присутствуют единичные выбросы, для целевого признака "reviewer_score" внесение изменений не возможно.

По условию соревнования строки не могут быть удалены, также данный подход приведет к значительной потере данных. Применение полной замены всех выбросов средними значениями для остальных рассмотренных призгаков может достаточно сильно исказить результат, предсказанный моделью.

В связи с этим выявленные выбросы в рамках данного исследования обрабатываться не будут.

#### 3.4 Исследование зависимостей в данных

##### 3.4.1 Рассмотрим особенности распределения признака "hotel_country" и его связь с целевой переменной

In [ ]:
fig_1 = plt.figure(figsize = (6, 3))
ax = fig_1.add_axes((1, 1, 1, 1))
pie = ax.pie(
    data['hotel_country'].value_counts(),
    autopct = " %1.1f%% ",
    startangle = 90,
    labels = data['hotel_country'].value_counts().index
)
title = ax.set_title('Распределение отелей по странам', fontsize = 16)
plt.show(pie)

***Вывод.***

В рассматриваемом датасете больше всего отелей расположено в Великобритании, примерно одинаковые показатели для Испании, Франции, Нидерландов (приблизительно по 11% от общего числа отелей), наименьшее число - в Австрии и Италии (приблизительно по 7% от общего числа отелей)

In [ ]:
# Рассмотрим, существует ли значительное различие в оценках для рассматриваемых стран 

# Возьмем данные для целевого признака из train выборки для построения сводной таблицы
reviewer_score_country = data[data['sample'] == 1].pivot_table(
    values = 'reviewer_score', 
    index = 'hotel_country', 
    aggfunc = ['count', 'mean'])

fig_2, ax = plt.subplots(1, 2, figsize = (15, 4))

barplot_1 = sns.barplot(
    data = reviewer_score_country,
    x = reviewer_score_country.index, 
    y = ('count', 'reviewer_score'),
    ax = ax[0]) 
ax[0].set_xlabel('Зависимость количества оценок от страны, в которой расположен отель', fontsize=11)

barplot_2 = sns.barplot(
    data = reviewer_score_country,
    x = reviewer_score_country.index, 
    y = ('mean', 'reviewer_score'),
    ax = ax[1])
ax[1].set_xlabel('Зависимость средней оценки от страны, в которой расположен отель', fontsize=11)
plt.ylim(8, 8.6) # ограничим диапазон оценки для большей наглядности


***Вывод.***

На основании построенных диаграмм можно сделать вывод о влиянии страны/города на целевой признак (данные для различных стран сильно отличаются). 

Так, при самом высоком показателе по количеству оценок Великобритания имеет самый низкий показатель средней оценки. Напротив, для Австрии количество оценок минимально, а средняя оценка самая высокая.

Полученные результаты могут быть обусловлены как субъективными особенностями рецензентов, так и объективными параметрами (особенностями климата стран, инфраструктурой и т.д.) 

##### 3.4.2 Выявим особенности распределения целевого показателя в зависимости от временного фактора

In [ ]:
# Рассмотрим, существует ли значительное различие в оценках в зависимости от месяца размещения отзыва

# Возьмем данные для целевого признака из train выборки для построения сводной таблицы
reviewer_score_month = data[data['sample'] == 1].pivot_table(
    values = 'reviewer_score', 
    index = 'review_month', 
    aggfunc = ['count', 'mean'])

fig_3, ax = plt.subplots(1, 2, figsize = (25, 10))

barplot_3 = sns.barplot(
    data = reviewer_score_month,
    x = reviewer_score_month.index, 
    y = ('count', 'reviewer_score'),
    ax = ax[0]) 
ax[0].set_xlabel('Зависимость количества оценок от месяца', fontsize = 16)
for p in barplot_3.patches:
    barplot_3.annotate(
        '{}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 14)

barplot_4 = sns.barplot(
    data = reviewer_score_month,
    x = reviewer_score_month.index, 
    y = ('mean', 'reviewer_score'),
    ax = ax[1])
ax[1].set_xlabel('Зависимость средней оценки от месяца', fontsize = 16)
for p in barplot_4.patches:
    barplot_4.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 16)
plt.ylim(8, 8.6) # ограничим диапазон оценки для большей наглядности

In [ ]:
# Рассмотрим, существует ли значительное различие в оценках в зависимости от дня недели размещения отзыва

# Возьмем данные для целевого признака из train выборки для построения сводной таблицы
reviewer_score_weekday = data[data['sample'] == 1].pivot_table(
    values = 'reviewer_score', 
    index = 'review_weekday', 
    aggfunc = ['count', 'mean'])

fig_4, ax = plt.subplots(1, 2, figsize=(25, 10))

barplot_5 = sns.barplot(
    data = reviewer_score_weekday,
    x = reviewer_score_weekday.index, 
    y = ('count', 'reviewer_score'),
    ax = ax[0]) 
ax[0].set_xlabel('Зависимость количества оценок от дня недели', fontsize = 16)
for p in barplot_5.patches:
    barplot_5.annotate(
        '{}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 14)

barplot_6 = sns.barplot(
    data = reviewer_score_weekday,
    x = reviewer_score_weekday.index, 
    y = ('mean', 'reviewer_score'),
    ax = ax[1])
ax[1].set_xlabel('Зависимость средней оценки от дня недели', fontsize = 16)
for p in barplot_6.patches:
    barplot_6.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 16)
plt.ylim(8, 8.6) # ограничим диапазон оценки для большей наглядности

***Вывод.***

Пиковые месяцы по количеству отзывов - май и июль-август, что можно объяснить оптимальным сочетанием стоимости туров и погодных условий (май) и периодом летних отпусков.

Что касается распределения по дням недели: наибольшее количество отзывов оставлено во вторник, далее по убыванию следуют воскресенье и понедельник; в качестве одного из предположений подобного распределения - рецензенты оставляют отзывы после поездок, большинство из которых может заканчиваться в выходные. Далее по количеству отзывов распологается среда - возможно добавляются однодневные деловые поездки в начале недели.

Средние оценки также отличаются по месяцам и соответсвенно по временам года: пиковые - зимой (максимум приходится на февраль), чуть ниже - весной, далее - лето, минимальные - осенью (минимум - на октябрь).

Распределение средней оценки по дням недели не так сильно различается, максимум приходится на вторник, что может быть связано с тем, что в этот день оставлено наибольшее число отзывов.


как и максимум кол-ва оценок, далее к середине недели достигается минимум, и затем снова начинается рост.

##### 3.4.3 Проверим, связаны ли между собой признаки "average_score" и "reviewer_score"

In [ ]:
# Создадим сводную таблицу со средними значениями признаков из train выборки
train_data_score = data[data['sample'] == 1].pivot_table(
    values = 'reviewer_score', 
    index = 'average_score', 
    aggfunc = 'mean').reset_index()

fig_5, ax = plt.subplots(figsize = (8, 6))

sns.kdeplot(train_data_score['reviewer_score'], ax = ax, label = 'reviewer_score')
sns.kdeplot(train_data_score['average_score'], ax = ax, label = 'average_score')
ax.set_title('Распределение среднего балла отеля и средних оценок', fontweight = 'bold', fontsize = 14)
ax.set_xlabel('Score')
ax.legend()

***Вывод.***

Исходя из полученных графиков, можно предположить, что характер их распределения близок к нормальному

In [ ]:
# Проверим гипотезу о нормальности распределения данных признаков 
# с помощью критерия Шапиро-Уилка

# Определяем гипотезы для проверки
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

alpha = 0.05

_, p = shapiro(train_data_score['reviewer_score'])
print('reviewer_score p-value = %.3f' % p)
if p > alpha:
    print(H0, '\n')
else:
    print(Ha, '\n')

_, p = shapiro(train_data_score['average_score'])
print('average_score p-value = %.3f' % p)
if p > alpha:
    print(H0)
else:
    print(Ha)

***Вывод.***

Подтверждение предположения о нормальном распределении рассматриваемых признаков с помощью критерия Шапиро-Уилка позволяет рассмотреть их взаимосвязь с точки зрения корреляции Пирсона

In [ ]:
display(train_data_score.corr())

***Вывод.***
На основе полученных данных можно сделать вывод о наличии очень сильной взаимосвязи для рассматриваемых признаков для тренировочной выборки. 

##### 3.4.4 Выявим особенности распределения целевого показателя в зависимости от национальности рецензента

In [ ]:
# Рассмотрим, существует ли значительное различие в оценках в зависимости от национальности рецензента

# Возьмем данные для целевого признака из train выборки для построения сводной таблицы
reviewer_score_nationality = data[data['sample'] == 1].pivot_table(
    values = 'reviewer_score', 
    index = 'reviewer_nationality', 
    aggfunc = ['count', 'mean'])

fig_6, ax = plt.subplots(2, 1, figsize = (15, 10))
fig_6. tight_layout (h_pad = 12) # отрегулируем интервал заголовков

barplot_7 = sns.barplot(
    data = reviewer_score_nationality,
    x = reviewer_score_nationality.index, 
    y = ('count', 'reviewer_score'),
    ax = ax[0]) 
ax[0].set_xlabel('Зависимость количества оценок от национальности рецензента', fontsize = 18)
ax[0].tick_params(axis = 'x', rotation = 45)
for p in barplot_7.patches:
    barplot_7.annotate(
        '{}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 14)

barplot_8 = sns.barplot(
    data = reviewer_score_nationality,
    x = reviewer_score_nationality.index, 
    y = ('mean', 'reviewer_score'),
    ax = ax[1])
ax[1].set_xlabel('Зависимость средней оценки от национальности рецензента', fontsize = 18)
ax[1].tick_params(axis = 'x', rotation = 45)
for p in barplot_8.patches:
    barplot_8.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 16)

***Вывод.***

На основе полученной диаграммы наглядно видно, что абсолютными лидерами по количесву оценок отелей являются рецензенты из Великобритании. Хотелось бы отметить, что и большинство рассматриваемых отелей также расположено в Великобритании.

Рецензенты из США, Австралии, Канады ставят отелям оценки выше, чем путешественники из европейских стран. 

Это может быть связано с тем, что в рамках данного исследования рассматриваются страны Европы, и возможно путешественники из указанных выше стран с наибольшими показателями менее требовательны к европейским отелям по сравнению с теми, кто живёт в Европе. 

Минимальные оценки - для рецензентов из Саудовской Аравии и ОАЭ, что может быть обусловлено значительными различиями в культуре, религии и особенностях инфраструктуры

##### 3.4.5 Рассмотрим, существует ли взаимосвязь между количеством слов  и целевым признаком

In [ ]:
fig_7, axes = plt.subplots(figsize = (15, 7))

sns.lineplot(
    data = data[data['sample'] == 1],
    x = 'reviewer_score',
    y = 'review_total_negative_word_counts',
    color = 'red', linewidth = 3,
    ax = axes
)
sns.lineplot(
    data = data[data['sample'] == 1],
    x = 'reviewer_score',
    y = 'review_total_positive_word_counts',
    color = 'black', linewidth = 3,
    ax = axes
)
axes.set_title('Взаимосвязь количества слов в отзыве и оценки', fontweight = 'bold', size = 16)
axes.set_xlabel('Score')
axes.set_ylabel('Total word count')
axes.set_xticks(np.arange(2, 11, 0.5))
axes.legend(['Negative', 'Word count', 'Positive', 'Word count'], loc = 'upper right', fontsize = 12)

***Вывод.***

В качестве общей тенденции, представленной на графике для негативных отзывов, чем ниже оценка за негативный отзыв, тем больше слов рецензенты употребляют при его написании.

С увеличением оценки за негативный отзыв, количество слов значительно снижается, что может быть связано с тем, что в отзыве отображена только его суть, без выраженной эмоциональной окраски, короткими, ближе к нейтральным, фразами. 

При низкой оценке за положительный отзыв, рецензенты склонны употреблять односложные слова, короткие предложения, без написания подробного отзыва. 

С увеличением оценки за положительный отзыв, путешественники склонны к более развернутому описанию, содержащему также описание впечатлений и эмоций.

##### 3.4.6 Рассмотри влияние информации, представленной в тегах, на целевой признак

In [ ]:
# Возьмем данные для целевого признака из train выборки для построения сводной таблицы
reviewer_score_purpose = data[data['sample'] == 1].pivot_table(
    values = 'reviewer_score', 
    index = 'purpose_trip', 
    aggfunc = ['count', 'mean'])

fig_8, ax = plt.subplots(2, 1, figsize = (15, 10))
fig_8. tight_layout (h_pad = 12) # отрегулируем интервал заголовков

barplot_9 = sns.barplot(
    data = reviewer_score_purpose,
    x = reviewer_score_purpose.index, 
    y = ('count', 'reviewer_score'),
    ax = ax[0]) 
ax[0].set_xlabel('Зависимость количества оценок от цели поездки', fontsize = 18)
ax[0].tick_params(axis = 'x', rotation = 45)
for p in barplot_9.patches:
    barplot_9.annotate(
        '{}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 14)

barplot_10 = sns.barplot(
    data = reviewer_score_purpose,
    x = reviewer_score_purpose.index, 
    y = ('mean', 'reviewer_score'),
    ax = ax[1])
ax[1].set_xlabel('Зависимость средней оценки от цели поездки', fontsize = 18)
ax[1].tick_params(axis = 'x', rotation = 45)
for p in barplot_10.patches:
    barplot_10.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 16)

***Вывод.***

По количеству оценок абсолютным лидером являются рецензенты, отправившиеся на отдых; значительно ниже данный показатель для деловых оценок.

Напротив, распределение средней оценки в зависмости от цели поездки более равномерно, но также незначительно преобладают рецензенты, путешествующие с целью отдыха.

In [ ]:
# Возьмем данные для целевого признака из train выборки для построения сводной таблицы
reviewer_score_travelers = data[data['sample'] == 1].pivot_table(
    values = 'reviewer_score', 
    index = 'travelers', 
    aggfunc = ['count', 'mean'])

fig_9, ax = plt.subplots(2, 1, figsize = (15, 10))
fig_9. tight_layout (h_pad = 12) # отрегулируем интервал заголовков

barplot_11 = sns.barplot(
    data = reviewer_score_travelers,
    x = reviewer_score_travelers.index, 
    y = ('count', 'reviewer_score'),
    ax = ax[0]) 
ax[0].set_xlabel('Зависимость количества оценок от состава путешествующих', fontsize=18)
ax[0].tick_params(axis = 'x', rotation = 45)
for p in barplot_11.patches:
    barplot_11.annotate(
        '{}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 14)

barplot_12 = sns.barplot(
    data = reviewer_score_travelers,
    x = reviewer_score_travelers.index, 
    y = ('mean', 'reviewer_score'),
    ax = ax[1])
ax[1].set_xlabel('Зависимость средней оценки от состава путешествующих', fontsize = 18)
ax[1].tick_params(axis = 'x', rotation = 45)
for p in barplot_12.patches:
    barplot_12.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 16)

***Вывод.***

Наибольшее количество оценок оставлено путешествующими в паре, значительно ниже (примерно в 2,3 раза) - для рецензентов, отправивщихся в поездку без сопровождения. Минимальный показатель по данному параметру - для путешествующих с друзьями.

Показатель средней оценки в зависимости от состава путешествующих распределен более равномерно, разброс в данных незначителен.

In [ ]:
# Возьмем данные для целевого признака из train выборки для построения сводной таблицы
reviewer_score_room = data[data['sample'] == 1].pivot_table(
    values = 'reviewer_score', 
    index = 'type_of_room', 
    aggfunc = ['count', 'mean'])

fig_10, ax = plt.subplots(2, 1, figsize = (15, 10))
fig_10. tight_layout (h_pad = 12) # отрегулируем интервал заголовков

barplot_13 = sns.barplot(
    data = reviewer_score_room,
    x = reviewer_score_room.index, 
    y = ('count', 'reviewer_score'),
    ax = ax[0]) 
ax[0].set_xlabel('Зависимость количества оценок от типа номера', fontsize = 18)
ax[0].tick_params(axis = 'x', rotation = 45)
for p in barplot_13.patches:
    barplot_13.annotate(
        '{}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 14)

barplot_14 = sns.barplot(
    data = reviewer_score_room,
    x = reviewer_score_room.index, 
    y = ('mean', 'reviewer_score'),
    ax = ax[1])
ax[1].set_xlabel('Зависимость средней оценки от типа номера', fontsize = 18)
ax[1].tick_params(axis = 'x', rotation = 45)
for p in barplot_14.patches:
    barplot_14.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 16)

***Вывод.***

Максимальное число отзывов оставлено рецензентами, остановившимися в номерах типа "double", далее - путешествующие, выбhавшие номера типов "standard" и "superior"; минимальное число отзывов - для группы рецензентов с типом номера "premier", что можно объяснить субъективными особенностями данной группы рецензентов, их социально-экономическим статусом.

Хотелось бы отметить, что распределение средней оценки в зависимости от типа номера практически равномерно; наивысший показатель - для номеров типа "premier", что обусловлено их повышенной комфортностью и комплектованием.  

In [ ]:
# Возьмем данные для целевого признака из train выборки для построения сводной таблицы
reviewer_score_nights = data[data['sample'] == 1].pivot_table(
    values = 'reviewer_score', 
    index = 'count_nights', 
    aggfunc = ['count', 'mean'])

fig_11, ax = plt.subplots(2, 1, figsize = (15, 10))
fig_11. tight_layout (h_pad = 12) # отрегулируем интервал заголовков

barplot_15 = sns.barplot(
    data = reviewer_score_nights,
    x = reviewer_score_nights.index, 
    y = ('count', 'reviewer_score'),
    ax = ax[0]) 
ax[0].set_xlabel('Зависимость количества оценок от длительности поездки', fontsize = 18)
ax[0].tick_params(axis = 'x', rotation = 45)
for p in barplot_15.patches:
    barplot_15.annotate(
        '{}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 14)

barplot_16 = sns.barplot(
    data = reviewer_score_nights,
    x = reviewer_score_nights.index, 
    y = ('mean', 'reviewer_score'),
    ax = ax[1])
ax[1].set_xlabel('Зависимость средней оценки от длительности поездки', fontsize = 18)
ax[1].tick_params(axis = 'x', rotation = 45)
for p in barplot_16.patches:
    barplot_16.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()),
    ha = 'center', va = 'bottom', fontsize = 16)

***Вывод.***

Наибольшее количество отзывов оставлено рецензентами, отправившимися в поездку на небольшой срок (от 1 до 5 ночей); при этом при увеличении срока пребывания в отеле количество отзывов начинается сокращаться.

Наивысшая средняя оценка была оставлена рецензентами, которые останавливались в отеле на длительный срок (масксимум данного показателя приходится на значение 28 ночей), то есть при выборе отеля рецензенты должны были быть скорее всего уверены в его положительных характеристиках, которые в дальнейшем подтвердились при проживании; при этом в целом распределение средней оценки в зависимости от длительности пребывания (для показателей от 1 до 25 ночей) достаточно равномерно

##### 3.4.7 Выявление взаимосвязи между целевой переменной и 'total_number_of_reviews_reviewer_has_given' 

In [ ]:
# Возьмем данные для целевого признака из train выборки
fig = sns.scatterplot(data = data[data['sample'] == 1], x = 'total_number_of_reviews_reviewer_has_given', y = 'reviewer_score')
fig.set_title('Зависимость средней оценки от количества прошлых отзывов', fontdict = {'fontsize':14})

***Вывод.***

Исходя из логических рассуждений, можно сделать предположение: если у отеля много отзывов, он, как правило, популярен и соответственно должен имеет высокие оценки.

Видим,отели с низкими прошлыми оценками до 4 чаще всего имеют кол-во прошлых отзывов до 50, с оценками 4-7 - до 100,

а те отели, у которых самое большое кол-во прошлых отзывов, имеют высокие оценки 8-10.

Есть отдельные точки, которые можно определить как выбросы.

Возможно имеет смысл превратить признак в категориальный, сгруппировать кол-во прошлых отзывов в интервалах до 50, 50-100, 100-150 и свыше 150.

##### 3.4.8 Выявление взаимосвязи между удаленностью отеля от центра города и целевым показателем

In [ ]:
reviewer_score_distance = data[data['sample'] == 1].pivot_table(
    values = 'distance_from_center',
    columns = 'reviewer_score',
    index = 'hotel_city')
reviewer_score_distance.columns = reviewer_score_distance.columns.astype('string')

fig_12, ax = plt.subplots(figsize = (10,7))    
heatmap = sns.heatmap(data=reviewer_score_distance, cmap = 'YlGnBu', 
                      cbar_kws = {'label': 'Расстояние до центра города'})
heatmap.set_title('Зависимость оценки от расстояния между отелем и центром для каждого города',
                  fontsize = 15, fontweight = 'bold', pad = 20);
heatmap.set_xlabel('reviewer score');
heatmap.set_ylabel('Название города')

***Вывод.***

На основании приведенной тепловой карты можно выявить для большинства показателей тенденцию, что чем выше оценка отеля, тем ближе он расположен к центру города.

#### 3.5 Масштабирование числовых признаков

В рамках данного исследования в качестве используемой модели предложена RandomForestRegressor (алгоритм случайного леса); в связи с этим нормализация и стандартизация признаков не требуется [C. Рашка - Python и машинное обучение, 2017, глава 4, стр. 118]

#### 3.6 Отбор признаков

In [ ]:
# На основании указанных признаков были получены необходимые данные,
# созданы новые признаки, в связи с этим удалим их 
data = data.drop(["hotel_address", "review_date", "hotel_name", "negative_review",
        "positive_review", "tags", "total_number_of_reviews_reviewer_has_given", "lat", "lng", "hotel_country"], axis = 1)

#### 3.7 Кодирование данных

In [ ]:
# Кодирование порядковых признаков
ord_enc = OrdinalEncoder()

data["groups_number_reviews_coder"] = ord_encoder.fit_transform(data[['groups_number_reviews']])


In [ ]:
# Кодирование OneHotEncoding для признаков, содержащих менее 10 уникальных значений

data['hotel_city'] = data['hotel_city'].str.lower()
encoder = ce.OneHotEncoder(cols = ['hotel_city']) 
type_bin = encoder.fit_transform(data['hotel_city'])
data = pd.concat([data, type_bin], axis = 1)

encoder = ce.OneHotEncoder(cols = ['purpose_trip']) 
type_bin = encoder.fit_transform(data['purpose_trip'])
data = pd.concat([data, type_bin], axis = 1)

encoder = ce.OneHotEncoder(cols = ['travelers']) 
type_bin = encoder.fit_transform(data['travelers'])
data = pd.concat([data, type_bin], axis = 1)

encoder = ce.OneHotEncoder(cols = ['review_year']) 
type_bin = encoder.fit_transform(data['review_year'])
data = pd.concat([data, type_bin], axis = 1)

encoder = ce.OneHotEncoder(cols = ['review_weekday']) 
type_bin = encoder.fit_transform(data['review_weekday'])
data = pd.concat([data, type_bin], axis = 1)

In [ ]:
# Двоичное кодирование для признаков, содержащих более 10 значений

data['reviewer_nationality'] = data['reviewer_nationality'].str.lower().str.replace(' ', '_')
bin_encoder = ce.BinaryEncoder(cols = ['reviewer_nationality']) 
type_bin = bin_encoder.fit_transform(data['reviewer_nationality'])
data = pd.concat([data, type_bin], axis = 1)

bin_encoder = ce.BinaryEncoder(cols = ['review_month']) 
type_bin = bin_encoder.fit_transform(data['review_month'])
data = pd.concat([data, type_bin], axis = 1)

bin_encoder = ce.BinaryEncoder(cols = ['type_of_room']) 
type_bin = bin_encoder.fit_transform(data['type_of_room'])
data = pd.concat([data, type_bin], axis = 1)

In [ ]:
# Удалим закодированные выше признаки
cols_to_del = ['hotel_city', 'purpose_trip', 'travelers', 'review_year',
                'review_weekday', 'reviewer_nationality', 'review_month',  
                'type_of_room', 'groups_number_reviews'
                ]
data = data.drop(cols_to_del, axis = 1)

In [ ]:
data.info()

#### 3.8 Анализ мультиколлинеарности

In [ ]:
# Выявим наличие сильно коррелирующих признаков, исключив целевую переменную и признак 'sample'
corr = data.drop(['sample', 'reviewer_score'], axis = 1).corr()
corr = corr[np.abs(corr) >= 0.8]

plt.subplots(figsize = (20,15))
heatmap = sns.heatmap(
    corr, 
    annot = True, 
    linewidths = 0.5, 
    vmin =-1, 
    vmax =1, 
    center = 0, 
    linecolor = 'grey')
heatmap.set_title('Корреляция признаков', fontweight = 'bold', size = 16)

In [ ]:
# Удалим по одному из пары сильно коррелирующих признаков
cols_to_del_corr = ["negative_review_neu", 'purpose_trip_2']
data = data.drop(cols_to_del_corr, axis = 1)

#### 3.9 Оценка значимости признаков

In [ ]:
# Числовые признаки
num_cols = ['average_score', 'review_total_negative_word_counts', 'total_number_of_reviews',
            'review_total_positive_word_counts', 'days_since_review','negative_review_neg',
            'negative_review_pos', 'negative_review_compound', 
            'positive_review_neg',  'positive_review_neu', 'positive_review_pos', 'positive_review_compound']

# Категориальные признаки
cat_cols = ['resident', 'no_negative_review', 'no_positive_review','with_pet', 'count_nights', 'submitted_mobile_device', 'distance_from_center',
            'hotel_city_1', 'hotel_city_2', 'hotel_city_3', 'hotel_city_4', 'hotel_city_5',
            'hotel_city_6', 'purpose_trip_1', 'purpose_trip_3', 'travelers_1', 'travelers_2',
            'travelers_3', 'travelers_4', 'travelers_5', 'travelers_6', 'review_year_1',
            'review_year_2', 'review_year_3', 'review_weekday_1', 'review_weekday_2',
            'review_weekday_3', 'review_weekday_4','review_weekday_5', 'review_weekday_6',
            'review_weekday_7', 'reviewer_nationality_0', 'reviewer_nationality_1', 
            'reviewer_nationality_2', 'reviewer_nationality_3', 'review_month_0',
            'review_month_1', 'review_month_2', 'review_month_3', 'type_of_room_0',
            'type_of_room_1','type_of_room_2', 'type_of_room_3', 'groups_number_reviews_coder']

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis = 1)
test_data = data.query('sample == 0').drop(['sample'], axis = 1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
y = y.astype('int')

# хи-квадрат

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index = cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
# Отсортируем полученный выше показатели для последующего отбора
imp_cat.sort_values()

In [ ]:
# anova

imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
# Удалим признаки, слабо влияющие на целевую переменную

features_to_drop = ['positive_review_neg', 'review_month_1', 'review_weekday_2',
                    'review_weekday_7', 'review_weekday_6', 'review_month_3', 
                    'type_of_room_1', 'review_weekday_1', 'review_weekday_4', 'review_month_2', 'with_pet',
                    'travelers_6', 'review_weekday_3', 'type_of_room_0', 'review_weekday_5', 'review_year_2',
                    'travelers_5', 'hotel_city_5', 'hotel_city_1', 'travelers_2', 'review_year_1',
                    'hotel_city_2', 'review_month_0', 'travelers_4',
                    'reviewer_nationality_1', 'review_year_3', 'type_of_room_2', 'purpose_trip_3',
                    'hotel_city_6', 'hotel_city_3', 'count_nights', 'resident', 'type_of_room_3', 'reviewer_nationality_2',
                    'hotel_city_4'] 

#### 3.10 Подготовка, обучение модели

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators = 100, verbose = 1, n_jobs = -1, random_state = RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))


In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis = 1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv_itog', index = False)
sample_submission.head(10)